<a href="https://colab.research.google.com/github/krupaltisgaonkar/pytorch-yolo/blob/main/YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLOv8 Training**
**Author:** Krupal Tisgaonkar

**Last updated:** January 18, 2025

---

This notebook helps you:
1. Upload or import a dataset from Google Drive.
2. Resize images and labels for YOLO format
3. Train a YOLOv8 model using the processed dataset.
4. Save and download the trained model.
5. You will be able to deploy on a local machine.

## **Step 1: Setup Environment**

In [ ]:
# Install required libraries
!pip install ultralytics

from google.colab import drive
from google.colab import files
import zipfile
import os
from PIL import Image

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Create necessary directories
os.makedirs("dataset", exist_ok=True)
os.makedirs("processed_dataset", exist_ok=True)

## **Step 2: Upload or Import Dataset**

1. Upload a zipped dataset manually.
2. Provide the path to a zipped dataset stored in Google Drive.

Structure your dataset as follows:

```
dataset/
├── images/
│   ├── train/
│   ├── val/
├── labels/
│   ├── train/
│   ├── val/
```

If your dataset isn't set like that, after this step, do Step A, otherwise don't do that step

> **NOTE**: Make sure to name your dataset dataset.zip

**Option 1: Upload zipped dataset manually**

In [ ]:
print("Upload your zipped dataset...")
uploaded = files.upload()
# Extract the uploaded dataset
if uploaded:
    for filename in uploaded.keys():
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("custom_data")
        print(f"Dataset extracted to: custom_data/")

**Option 2: Import from Google Drive**

In [2]:
drive_dataset_path = "/content/drive/MyDrive/dataset/YOLO/dataset.zip"  # Replace with your Google Drive dataset path
if os.path.exists(drive_dataset_path):
    with zipfile.ZipFile(drive_dataset_path, 'r') as zip_ref:
        zip_ref.extractall("custom_data")
    print(f"Dataset extracted from Google Drive to: custom_data/")

Dataset extracted from Google Drive to: custom_data/


### Step A: Make Dataset Compatible

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

## **Step 3: Resize Images to 640x640**

Resize all images in the `train` and `val` directories to `640x640`.

In [4]:
# Function to resize images
def resize_images(input_dir, output_dir, new_size=(640, 640)):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(input_dir, filename)
            img = Image.open(img_path)
            img_resized = img.resize(new_size)
            img_resized.save(os.path.join(output_dir, filename))
    print(f"Images resized and saved to {output_dir}.")

# Resize train and val images
resize_images("data/train/images", "processed_dataset/train/images")
resize_images("data/validation/images", "processed_dataset/val/images")

Images resized and saved to processed_dataset/train/images.
Images resized and saved to processed_dataset/val/images.


## **Step 4: Resize Labels**

Adjust YOLO-format labels to align with the new image size. For the

In [ ]:
!wget -O /content/resize_labels.py https://raw.githubusercontent.com/krupaltisgaonkar/pytorch/refs/heads/main/scripts/resize_labels.py

!python resize_labels.py --input_label_dir data/train/labels \
                 --input_image_dir data/train/images \
                 --output_label_dir processed_dataset/train/labels \
                 --new_size 640
!python resize_labels.py --input_label_dir data/validation/labels \
                 --input_image_dir data/validation/images \
                 --output_label_dir processed_dataset/val/labels/ \
                 --new_size 640

## **Step 5: Train YOLOv8**

Use the resized dataset to train a YOLOv8 model.

In [ ]:
%cd /content/
!mkdir datasets
!cp -r /content/processed_dataset /content/datasets

Create a data.yaml file for YOLO

In [ ]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/processed_dataset',
      'train': 'train/images',
      'val': 'val/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml("custom_data/classes.txt", "/content/data.yaml")

print('\nFile contents:\n')
!cat /content/data.yaml

Start Training!



> **NOTE:** Your model might need more epochs if it has less images

After training this will already download the model.





In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=2500 imgsz=640 patience=2501

# Create "my_model" folder to store model weights and train results
!mkdir /content/fish_detect
!cp /content/runs/detect/train/weights/best.pt /content/fish_detect/my_model.pt
!cp -r /content/runs/detect/train /content/fish_detect

# Zip into "my_model.zip"
%cd fish_detect
!zip /content/fish_detect.zip model.pt
!zip -r /content/fish_detect.zip train
%cd /content
files.download('/content/fish_detect.zip')

## Step 6: **Deploy on Local Machine**

Download Anaconda Prompt, or you can just create a virtual environment by yourself.

Name it whatever you want I named mine `yolo-env1`. Also ensure it is python 3.12.

For Anaconda, I used this command:

```
conda create --name yolo-env1 python=3.12 -y
conda activate yolo-env1
```
Now install Ultralytics by using

```
pip install ultralytics
```

If you have a NVIDIA GPU, CUDA enabled on your local computer, use this command to install the packages

```
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

If you **DO NOT** have a GPU, or the proper GPU, use this command to download the CPU version:

```
pip install torch torchvision torchaudio
```

Now to run this model, you will have to download the code from my github page.

You can use either:

```
curl -o model_main.py https://raw.githubusercontent.com/krupaltisgaonkar/pytorch/refs/heads/main/scripts/model_main.py
```

or

```
wget -O model_main.py https://raw.githubusercontent.com/krupaltisgaonkar/pytorch/refs/heads/main/scripts/model_main.py
```

**REMEMBER TO CHANGE YOUR CUSTOM MODELS VARIABLE'S PATHS TO THE PATHS OF YOUR MODELS!!! IN THE CODE**


Run the command:

Use a YOLO Predefined Model (v5, v8, v11) with Webcam

```
python model_main.py --model v8 --webcam 0
```

Use a YOLO Predefined Model with an Image or Video or Directory

```bash
python model_main.py --model v5 --input ./data/image.jpg
python model_main.py --model v11 --input ./data/video.mp4
python model_main.py --model v11 --input ./data/
```
Use your custom model
```
python model_main.py --model custom1 --input ./data/image.jpg

```

## **Errors**

If you have question, go to the github repository, and under issues, put an issue.